In [ ]:
# /content/drive/MyDrive/Colab Notebooks/pokedata.zip
from google.colab import drive
drive.mount('/content/drive')
# !mkdir dataset
import PIL  # python 의 이미지 전처리 라이브러리
import glob # 대용량, 대규모 파일을 다룰때 사용
import os # os, path 관련 조작, 변경 , 파일 삭제 등
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/pokedata.zip -d dataset

In [ ]:
import PIL  # python의 이미지 전처리 라이브러리
import glob # 대용량, 대규모 파일을 다룰 때 사용
import os   # os, path 관련 조작, 변경, 파일 삭제 등
dataset_folder_path = '/content/dataset'
category_cnt = len(os.listdir(dataset_folder_path))
print(os.listdir(dataset_folder_path))

['water', 'grass']


In [ ]:
# glob.glob(dataset_folder_path+'/*/*')
# 파일의 경로+/* 를 입력하면 해당 폴더 하위 경로 상에 있는 모든 파일을
# 리스트로 리턴
# ex) dataset_folder_path+'/*/*' -> 하위 폴더의 하위 데이터 전체 리턴
import numpy as np
x = []
y = []

for label, folder in enumerate(os.listdir(dataset_folder_path)):
  # print(dataset_folder_path +'/'+ folder) 좋은 코드가 아님
  # print(os.path.join(dataset_folder_path, folder)) # 유지보수 좋은 코드
  folder_full_path = os.path.join(dataset_folder_path, folder)
  all_files = glob.glob(folder_full_path + '/*)')
  # print(folder_full_path)
  # print(all_files)
  for idx, file_full_path in enumerate(all_files):
    image = PIL.Image.open(file_full_path)
    image = image.resize((64, 64))
    image = image.convert('RGB')
    data = np.asarray(image)
    x.append(data)
    y.append(label)
    if idx % 10 == 0:
      print(idx, '/', len(all_files))

0 / 130
10 / 130
20 / 130
30 / 130
40 / 130
50 / 130
60 / 130
70 / 130
80 / 130
90 / 130
100 / 130
110 / 130
120 / 130
0 / 174
10 / 174
20 / 174
30 / 174
40 / 174
50 / 174
60 / 174
70 / 174
80 / 174
90 / 174
100 / 174
110 / 174
120 / 174
130 / 174
140 / 174
150 / 174
160 / 174
170 / 174


In [ ]:
x = np.array(x)
y = np.array(y)
print('카테고리 갯수', np.bincount(y))
print('전처리 확인', x.shape)

카테고리 갯수 [130 174]
전처리 확인 (304, 64, 64, 3)


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
# 개발 팁, 전처리 시간이 너무 오래 걸린다면(데이터가 늘어 나서)
# xy = (x_train, x_test, y_train, y_test)
# np.save('/content/drive/MyDrive/preprocessing_data.npy', xy)

In [ ]:
# x_train, x_test, y_train, y_test = np.load('/content/drive/MyDrive/preprocessing_data.npy')

In [ ]:
x_train = x_train / 255.
x_test = x_test / 255.

In [ ]:
# 구글넷 실습(오전에 진행한)
#
# import tensorflow as tf
# from tensorflow.keras import layers, models

# 개선의 여지 해상도 수정 (전처리도 수정해야 함)
input_layer = layers.Input(shape = (64,64,3), name='start_layer')


# 개선의 여지 conv2d 수정. 성능이 매우 잘 나온다면 모델 다이어트
# 성능이 안나온다면 channel 수, 레이어 추가
tower_1 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)

tower_2 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_2 = layers.Conv2D(64, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2)

tower_3 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_3 = layers.Conv2D(64, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_3)

tower_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(input_layer)
tower_4 = layers.Conv2D(64, kernel_size=(1,1) , padding='same' , activation='relu')(tower_4)

concat_layer = layers.concatenate([tower_1, tower_2,tower_3,tower_4], axis=3)

tower_2_1 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)

tower_2_2 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_2 = layers.Conv2D(120, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2_2)

tower_2_3 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_3 = layers.Conv2D(120, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_2_3)

tower_2_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(concat_layer)
tower_2_4 = layers.Conv2D(120, kernel_size=(1,1) , padding='same' , activation='relu')(tower_2_4)

final_concat_layer = layers.concatenate([tower_2_1, tower_2_2,tower_2_3,tower_2_4], axis=3)
average_pooling_layer = layers.AveragePooling2D(pool_size=(16,16),strides=(16,16))(final_concat_layer)
flat_layer = layers.Flatten()(average_pooling_layer)

# 개선의 여지 - Dense 층 추가
output_layer = layers.Dense(1 , activation = 'sigmoid')(flat_layer)

google_net_model = models.Model(inputs = input_layer, outputs = output_layer)
google_net_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start_layer (InputLayer)    [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 conv2d_3 (Conv2D)           (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 3)            0         ['start_layer[0][0]']         
 D)                                                                                           

In [ ]:
# 개선 여지 hyper parameter tuning
# ex) tf.keras.optimizers.Adam()

google_net_model.compile(loss='binary_crossentropy',
                         optimizer='adam', metrics=['accuracy'])

In [ ]:
check_point_path = '/content/drive/MyDrive/ourmodel_checkpoint'
mc = tf.keras.callbacks.ModelCheckpoint(check_point_path,
                                   monitor='val_loss',
                                   save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                      patience=4)


In [ ]:
google_net_model.fit(x_train, y_train, batch_size=32,
                     epochs=20,
                     callbacks=[mc, es],
                     validation_data=(x_test, y_test))


Epoch 1/20
8/8 [==============================] - 132s 16s/step - loss: 0.7448 - accuracy: 0.4979 - val_loss: 0.6808 - val_accuracy: 0.5574
Epoch 2/20
8/8 [==============================] - 126s 16s/step - loss: 0.6828 - accuracy: 0.5802 - val_loss: 0.6639 - val_accuracy: 0.7377
Epoch 3/20
8/8 [==============================] - 126s 16s/step - loss: 0.6375 - accuracy: 0.6955 - val_loss: 0.6018 - val_accuracy: 0.6557
Epoch 4/20
8/8 [==============================] - 128s 16s/step - loss: 0.5498 - accuracy: 0.7572 - val_loss: 0.4741 - val_accuracy: 0.8525
Epoch 5/20
8/8 [==============================] - 125s 16s/step - loss: 0.4964 - accuracy: 0.7695 - val_loss: 0.6891 - val_accuracy: 0.6721
Epoch 6/20
8/8 [==============================] - 126s 16s/step - loss: 0.5505 - accuracy: 0.7243 - val_loss: 0.4964 - val_accuracy: 0.7705
Epoch 7/20
8/8 [==============================] - 125s 16s/step - loss: 0.5215 - accuracy: 0.7490 - val_loss: 0.4948 - val_accuracy: 0.7541
Epoch 8/20
8/8 [====

In [ ]:
google_net_model.load_weights('/content/drive/MyDrive/ourmodel_checkpoint')

In [ ]:
from google.colab import files
f = files.upload()

Saving wailord.png to wailord.png


In [ ]:
file_name=list(f.keys())[0]
test_img = PIL.Image.open(file_name)
test_img = test_img.resize((64, 64))
test_img = test_img.convert('RGB')
data = np.asarray(test_img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor/255.
test_image_tensor.shape


(1, 64, 64, 3)

In [ ]:
result = google_net_model.predict(test_image_tensor)
result[0][0]
if result[0][0] > 0.5:
  print(f'{result[0][0] * 100}% 확률로 풀타입')
else:
  print(f'{(1 - result[0][0]) * 100}% 확률로 물타입')

1/1 [==============================] - 0s 119ms/step
99.38043090514839% 확률로 물타입


성능 개선 (모델 수정, 데이터, 하이퍼메터 튜닝)

1. 데이터
2. 모델
3. 하이퍼 파라미터 수정 (우선 순위 최하)